## Imports

In [3]:
import re
import json
from tqdm import tqdm
from bs4 import BeautifulSoup
import bs4
import os
import sys

In [4]:
# Python has a recursion limit of 3K.
# We raise it up to 10K
sys.setrecursionlimit(10000)

In [5]:
DIR_RAW_DATA_PATH = './data/endata/'
DIR_CLEAN_DATA = './data/endata_new_clean/'
JSON_TRAIN_CORPUS = './data/wiki_html_all.json'

## MAIN

In [7]:
# Open Train Corpus file
with open(JSON_TRAIN_CORPUS,'w')as g:
    # foreach 
    for root, dirs, files in os.walk(DIR_RAW_DATA_PATH):
        print(root)
        print(dirs)
        print(files)
        print('')

        """ for dir in tqdm(dirs):
            sub_DIR_RAW_DATA_PATH = os.path.join(root,dir)
            for sub_root,sub_dirs,sub_files in os.walk(sub_DIR_RAW_DATA_PATH):
                # foeach file in dir
                for f in tqdm(sub_files):
                    #index = f.split('.')[0]
                    path = os.path.join(sub_root,f)
                    clean_path = DIR_CLEAN_DATA+'_clean.html'
                    cleaner = HTMLCleaner(path,clean_path)
                    cleaner.clean_html()
                    #cleaner.clean_tree()
                    storer = HTMLStorer(input_file=None,html=cleaner.line)
                    g.write(json.dumps(storer.data,ensure_ascii=False)+'\n') """


./data/endata/
3333
['auto', 'book', 'camera', 'job', 'movie', 'nbaplayer', 'restaurant', 'university']
4444
[]
./data/endata/auto
3333
['auto-aol(2000)', 'auto-autobytel(2000)', 'auto-automotive(1999)', 'auto-autoweb(2000)', 'auto-carquotes(2000)', 'auto-cars(657)', 'auto-kbb(2000)', 'auto-motortrend(1267)', 'auto-msn(2000)', 'auto-yahoo(2000)']
4444
[]
./data/endata/auto\auto-aol(2000)
3333
[]
4444
['0000.htm', '0001.htm', '0002.htm', '0003.htm', '0004.htm', '0005.htm', '0006.htm', '0007.htm', '0008.htm', '0009.htm', '0010.htm', '0011.htm', '0012.htm', '0013.htm', '0014.htm', '0015.htm', '0016.htm', '0017.htm', '0018.htm', '0019.htm', '0020.htm', '0021.htm', '0022.htm', '0023.htm', '0024.htm', '0025.htm', '0026.htm', '0027.htm', '0028.htm', '0029.htm', '0030.htm', '0031.htm', '0032.htm', '0033.htm', '0034.htm', '0035.htm', '0036.htm', '0037.htm', '0038.htm', '0039.htm', '0040.htm', '0041.htm', '0042.htm', '0043.htm', '0044.htm', '0045.htm', '0046.htm', '0047.htm', '0048.htm', '0049.h